# User Behavior Analysis

As engineers, product managers, and marketers, we know our product's value propositions the way we know our own birthdays. However, what's obvious to us is not always obvious to our users, causing unintentional dishonesty in qualitative studies, and even more difficult to identify with ony a basic understanding of user behavior on our app.

The goal of this project is to turn usage data into something tangible and actionable. It's not just about *what* the magic moment is for the user but *when* and *how*. A common example is Facebook-- their growth team mined and analyzed user data to identify that users find value in their product when they connect with ten friends within seven days.

![alt-text](/Users/allisonkelly/Documents/Notebooks/user_behavior_analysis/images/facebook-aha.png)

The illustration above shows the sweet spot between users who performed the action and retained vs. users who retained that performed the action. This is the tipping point for most users-- when the *what* becomes the *how many*. After finding out *when* the user must perform the action, we'll run validation tests and experiments to prove our hypothesis.

### Problem Statement

1. What is/are our most important feature(s) based on user behavior?
2. How many times must a user take advantage of that/those features to be retained?
3. Within what time frame must the user adopt the feature to be retained?

## Imports

Where can I find this file

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.getcwd()

'/Users/allisonkelly/Documents/Notebooks/user_behavior_analysis/notebooks'

# Data

Each of the following four dataframes represents a section of user behavior, pulled from our MySQL database. 

### Column Descriptions

- team_id: unique identifier for teams, can have multiple users
- account_start_date: date team account was created
- trial_end_date: end date of trial period
- mocks_count: count of projects
- entities_count: count of entities used across all projects
- intents_count: count of intents used across all projects 
- variables_count: count of variables used across all projects
- last_login: date of last login (by any member of the team)
- sub_end: date of subscrition end
- stripe_plan: name of subscription
- members: count of team members that are editors
- reviewers: count of team members that are reviewers
- testers: count of team members that are testers

In [3]:
raw_df1 = pd.read_csv("/Users/allisonkelly/Documents/Notebooks/user_behavior_analysis/data/raw/feature_counts_grouped_by_team.csv", 
                      names=['team_id', 'account_start_date', 'trial_end_date', 'mocks_count', 'entities_count', 'intents_count', 'variables_count'],
                     header=0)

In [4]:
len(raw_df1)

16888

In [5]:
raw_df2 = pd.read_csv("/Users/allisonkelly/Documents/Notebooks/user_behavior_analysis/data/raw/team_last_login_subsc_end_date.csv", 
                      names = ['team_id', 'last_login', 'sub_end', 'stripe_plan'], header=0)

raw_df2['team_id'] = raw_df2['team_id'].astype(object)

In [6]:
len(raw_df2)

18006

In [7]:
raw_df3 = pd.read_csv("/Users/allisonkelly/Documents/Notebooks/user_behavior_analysis/data/raw/mocks_comments_user_tests.csv")


In [8]:
raw_df3.head()

,id,team_id,no_comment,no_user_tests
0,0178dee0-aa06-11e7-8691-c9fce968bbcf,1,0,0
1,04c2f950-2fac-11e9-819b-432f8134e172,1,0,0
2,06855dd0-3923-11e7-be67-1174531b58f7,1,0,0
3,0d18ddd0-6d91-11e8-b4c3-17d192a9e275,1,0,0
4,1001da80-b7a5-11e9-ace5-69ef697942ba,1,0,0


In [9]:
raw_df3 = raw_df3.groupby(['team_id']).sum().reset_index()

In [10]:
raw_df4 = pd.read_csv("/Users/allisonkelly/Documents/Notebooks/user_behavior_analysis/data/raw/team_user_roles.csv")

In [11]:
len(raw_df4)

18580

In [34]:
merge_df = raw_df1.merge(raw_df2,on='team_id').merge(raw_df4,on='team_id')
merge_df.head()

,team_id,account_start_date,trial_end_date,mocks_count,entities_count,intents_count,variables_count,last_login,sub_end,stripe_plan,members,reviewers,testers
0,1,2017-01-10 15:45:21,NaN,73,7,41,27,2020-12-05 23:24:18,NaN,pro-plan,7,0,8
1,4,2017-01-10 16:09:13,2017-01-28 10:08:26,1,0,0,0,2019-05-14 16:52:00,NaN,NaN,0,0,0
2,5,2017-01-10 16:15:43,2017-01-28 10:08:26,1,0,0,0,NaN,NaN,NaN,0,0,0
3,6,2017-01-10 16:45:29,2017-01-28 10:08:26,1,0,0,0,NaN,NaN,NaN,0,0,0
4,7,2017-01-10 17:07:54,2017-01-28 10:08:26,1,0,0,0,NaN,NaN,NaN,0,0,0


In [35]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16372 entries, 0 to 16371
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   team_id             16372 non-null  object
 1   account_start_date  16372 non-null  object
 2   trial_end_date      15778 non-null  object
 3   mocks_count         16372 non-null  int64 
 4   entities_count      16372 non-null  int64 
 5   intents_count       16372 non-null  int64 
 6   variables_count     16372 non-null  int64 
 7   last_login          9126 non-null   object
 8   sub_end             934 non-null    object
 9   stripe_plan         1147 non-null   object
 10  members             16372 non-null  int64 
 11  reviewers           16372 non-null  int64 
 12  testers             16372 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 1.7+ MB


In [36]:
merge_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
mocks_count,16372.0,3.478256,26.287360,0.0,1.0,1.0,2.0,1381.0
entities_count,16372.0,2.348400,86.932887,0.0,0.0,0.0,0.0,8454.0
intents_count,16372.0,15.299841,686.188935,0.0,0.0,0.0,0.0,63619.0
variables_count,16372.0,8.609822,417.472297,0.0,0.0,0.0,0.0,45259.0
members,16372.0,0.146592,1.601763,0.0,0.0,0.0,0.0,129.0
reviewers,16372.0,0.071891,0.971413,0.0,0.0,0.0,0.0,54.0
testers,16372.0,0.162106,2.140844,0.0,0.0,0.0,0.0,113.0


# Data Cleaning

Retention as activity or churn?

Break down by plan type
- Free
- Pro
- Enterprise


has_invited
mocks - continuous
Free to paid
Paid to churn
free to churned
separate freemium users

In [37]:
enterprise_plans = ["aarp-yearly","allstate-plan1","bms-yearly","botmock15","centurylink-monthly","coned-plan",
                    "fleishman-yearly","plan_F7HeCobBL8LyDk","plan_FER0iNKmeqQuob","plan_H8jHSUawWHiNUl",
                    "plan-novartis","price_1HkCSdAJmWSUW5DJRPUifM1H","prod_FbBc8TFEZPLW47",'prod_FeUn3vSkQk6Rec',
                    'prod_GlpnGQpRGsGeyd','promero-monthly','vodafone-nz']

In [38]:
def check_enterprise(dfObj, listOfValues):
    ''' Check if given elements exists in Series or not.
        It returns a list of boolean values'''
    results = []
    for value in dfObj.values:
        # Check if the element exists in dataframe values
        if value in listOfValues:
            results.append(1)
        else:
            results.append(0)
    # Returns a list of true/false      
    return results

In [39]:
merge_df['is_enterprise'] = pd.Series(check_enterprise(merge_df['stripe_plan'], enterprise_plans))

In [40]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16372 entries, 0 to 16371
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   team_id             16372 non-null  object
 1   account_start_date  16372 non-null  object
 2   trial_end_date      15778 non-null  object
 3   mocks_count         16372 non-null  int64 
 4   entities_count      16372 non-null  int64 
 5   intents_count       16372 non-null  int64 
 6   variables_count     16372 non-null  int64 
 7   last_login          9126 non-null   object
 8   sub_end             934 non-null    object
 9   stripe_plan         1147 non-null   object
 10  members             16372 non-null  int64 
 11  reviewers           16372 non-null  int64 
 12  testers             16372 non-null  int64 
 13  is_enterprise       16372 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 1.9+ MB


Out of 16,372 users, 15,225 have accounts that are not associated with a subscription. There are 21 accounts on Enterprise plans and 1,126 accounts associated with a Pro plan.

In [41]:
print(f'Accounts with no paid subscription: {sum(merge_df.stripe_plan.isnull())}')
print(f'Accounts on Enterprise plans: {sum(merge_df.is_enterprise)}')
print(f'Accounts on Pro plans: {16372 - 15225 - 21}')

Accounts with no paid subscription: 15225
Accounts on Enterprise plans: 21
Accounts on Pro plans: 1126


In [42]:
# Swapping null values for "free" to indicate no subscription
merge_df.stripe_plan.fillna('free', inplace=True)

In [43]:
date_cols = ['account_start_date', 'trial_end_date', 'last_login', 'sub_end']
for col in date_cols:
    merge_df[col] = merge_df[col].astype('datetime64')

In [44]:
len(merge_df) - sum(merge_df.sub_end.isnull())

934

In [45]:
merge_df.head()

,team_id,account_start_date,trial_end_date,mocks_count,entities_count,intents_count,variables_count,last_login,sub_end,stripe_plan,members,reviewers,testers,is_enterprise
0,1,2017-01-10 15:45:21,NaT,73,7,41,27,2020-12-05 23:24:18,NaT,pro-plan,7,0,8,0
1,4,2017-01-10 16:09:13,2017-01-28 10:08:26,1,0,0,0,2019-05-14 16:52:00,NaT,free,0,0,0,0
2,5,2017-01-10 16:15:43,2017-01-28 10:08:26,1,0,0,0,NaT,NaT,free,0,0,0,0
3,6,2017-01-10 16:45:29,2017-01-28 10:08:26,1,0,0,0,NaT,NaT,free,0,0,0,0
4,7,2017-01-10 17:07:54,2017-01-28 10:08:26,1,0,0,0,NaT,NaT,free,0,0,0,0


In [46]:
free_df = merge_df.loc[merge_df['stripe_plan'] == 'free']
paid_df = merge_df.loc[merge_df['stripe_plan'] != 'free']

In [47]:
paid_df.head()

,team_id,account_start_date,trial_end_date,mocks_count,entities_count,intents_count,variables_count,last_login,sub_end,stripe_plan,members,reviewers,testers,is_enterprise
0,1,2017-01-10 15:45:21,NaT,73,7,41,27,2020-12-05 23:24:18,NaT,pro-plan,7,0,8,0
51,57,2017-01-11 12:48:38,NaT,1,0,0,0,2020-08-26 00:13:38,2017-02-15 07:07:30,solo-plan,0,0,0,0
75,82,2017-01-14 22:08:41,NaT,6,0,0,0,NaT,NaT,starter-plan,2,0,0,0
94,101,2017-01-22 09:14:29,NaT,1,0,0,0,NaT,2017-03-19 06:54:32,starter-plan,4,0,0,0
138,151,2017-02-11 01:29:34,2017-02-26 01:29:38,1,0,0,0,NaT,2017-02-26 01:29:38,starter-plan,2,0,0,0


In [48]:
len(paid_df)

1147

In [49]:
paid_df['account_start_date'] = pd.to_datetime(paid_df.account_start_date)

In [50]:
paid_df['sub_end'] = pd.to_datetime(paid_df['sub_end'])

In [51]:
paid_df['churned'] = np.where(paid_df.sub_end <= pd.datetime.now(), 1, 0)

In [52]:
paid_df['churned'].value_counts()

1    930
0    217
Name: churned, dtype: int64

In [53]:
paid_df.to_csv('paid.csv')